In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glasses.nn.models.classification.fishnet import *
import torch
import torch.nn as nn

In [3]:
from glasses.nn.blocks import Conv2dPad, ConvBnAct


In [6]:
ConvBnAct(32, 64, kernel_size=7, stride=2)

ConvBnAct(
  (conv): Conv2dPad(32, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU()
)

In [14]:
FishNet.fishnet99()

FishNet(
  (encoder): FishNetEncoder(
    (gate): Sequential(
      (0): ConvBnAct(
        (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (1): ConvBnAct(
        (conv): Conv2dPad(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (2): ConvBnAct(
        (conv): Conv2dPad(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (tail): ModuleList(
      (0): FishNetTailBlock(
        (block): Sequential(
          (0): FishNetBottleNeck(
  

In [19]:
from torchsummary import summary
model = FishNet.fishnet99()

summary(model.cuda(), (3,224,224))

torch.Size([2, 768, 14, 14])
torch.Size([2, 512, 28, 28])
torch.Size([2, 320, 56, 56])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
         Conv2dPad-4         [-1, 32, 112, 112]           9,216
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              ReLU-6         [-1, 32, 112, 112]               0
         Conv2dPad-7         [-1, 64, 112, 112]          18,432
       BatchNorm2d-8         [-1, 64, 112, 112]             128
              ReLU-9         [-1, 64, 112, 112]               0
        MaxPool2d-10           [-1, 64, 56, 56]               0
      BatchNorm2d-11           [-1, 64, 56, 56]             128
             ReLU-12           [-1, 64, 56, 56]               0
        Conv2dPa

(tensor(16634312), tensor(16634312))